In [21]:
metrics = {
    "profitability": {
        "class_weight": 0.35,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (0.4, float("inf")),
                    (0.35, 0.4),
                    (0.3, 0.35),
                    (0.25, 0.3),
                    (0.2, 0.25),
                    (0.15, 0.2),
                    (0.1, 0.15),
                    (0.05, 0.1),
                    (float("-inf"), 0.05),
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.65,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "debt_to_equity": {
                "lower_is_better": True,
                "thresholds": [
                    (0, 0.4),
                    (0.4, 0.8),
                    (0.8, 1.2),
                    (1.2, 1.6),
                    (1.6, 2),
                    (2, 2.5),
                    (2.5, 3),
                    (3, 4),
                    (4, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (0, float("-inf")),
                    (0.4, 0.8),
                    (0.8, 1.2),
                    (1.2, 1.6),
                    (1.6, 2),
                    (2, 2.5),
                    (2.5, 3),
                    (3, 4),
                    (4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
}


In [22]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics

    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        credit_ratings = [
            (1.5, "Aaa"),
            (2.5, "Aa"),
            (3.5, "A"),
            (4.5, "Baa"),
            (5.5, "Ba"),
            (6.5, "B"),
            (7.5, "Caa"),
            (8.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.weighted_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.weighted_score)

In [24]:
ratios = {
    'oper_margin': 2.0,
    'debt_to_equity': 2,
    'tot_debt_to_ebitda': 9.5,
    'ebitda_to_tot_int_exp': 0.6,
}

model = CreditRatingCalculator(metrics)
model.calculate_credit_rating(ratios)
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.weighted_score}")
print(f"Credit Rating: {model.credit_rating}")

Class Scoring: {'profitability': 1.0, 'leverage_coverage': 7.1}
Credit Score: 4.965
Credit Rating: Ba


In [26]:
import numpy as np

def forecast_ratios(ratios, priors, confidence_levels, years):
    # Convert ratios, priors, and confidence levels to numpy arrays
    ratios_array = np.array(list(ratios.values()))
    priors_array = np.array(priors)
    confidence_levels_array = np.array(confidence_levels)

    # Calculate the weights based on confidence levels
    weights = confidence_levels_array / np.sum(confidence_levels_array)

    # Calculate the posterior estimates
    posterior_estimates = weights * priors_array + (1 - weights) * ratios_array

    # Generate forecasts for the specified years
    forecasts = {}
    for year in range(1, years + 1):
        forecast_year = {}
        for i, ratio_name in enumerate(ratios.keys()):
            forecast_year[ratio_name] = posterior_estimates[i] * (1 + priors_array[i]) ** year
        forecasts[year] = forecast_year

    return forecasts

# Example usage
ratios = {
    'oper_margin': 2.0,
    'debt_to_equity': 2,
    'tot_debt_to_ebitda': 9.5,
    'ebitda_to_tot_int_exp': 0.6,
}

priors = [0.05, 0.1, 0.02, 0.08]  # Prior beliefs for each ratio
confidence_levels = [0.8, 0.6, 0.9, 0.7]  # Confidence levels for each prior

years = 4  # Number of years to forecast

forecasted_ratios = forecast_ratios(ratios, priors, confidence_levels, years)

# Print the forecasted ratios
for year, forecast in forecasted_ratios.items():
    print(f"Year {year} Forecast:")
    for ratio_name, ratio_value in forecast.items():
        print(f"{ratio_name}: {ratio_value:.2f}")
    print()

Year 1 Forecast:
oper_margin: 1.55
debt_to_equity: 1.78
tot_debt_to_ebitda: 6.79
ebitda_to_tot_int_exp: 0.52

Year 2 Forecast:
oper_margin: 1.63
debt_to_equity: 1.96
tot_debt_to_ebitda: 6.92
ebitda_to_tot_int_exp: 0.56

Year 3 Forecast:
oper_margin: 1.71
debt_to_equity: 2.16
tot_debt_to_ebitda: 7.06
ebitda_to_tot_int_exp: 0.60

Year 4 Forecast:
oper_margin: 1.80
debt_to_equity: 2.37
tot_debt_to_ebitda: 7.20
ebitda_to_tot_int_exp: 0.65

